In [1]:
import requests
import datetime

In [2]:
# your AgroDataCube access token
token = "<your token here>"

In [3]:
# year to retrieve data for from the AgroDataCube, should be in available range
year = 2016

In [4]:
# read field at a specific point and for a specific year (2016)
fields_url = "http://agrodatacube.wur.nl/api/v1/rest/fields?epsg=28992&year={0}&geometry=POINT(200000 500000)&page_size=1000&page_offset=0".format(year)
header_token = {"token":token}
fields_data = requests.get(fields_url, headers=header_token)
fields = fields_data.json()

In [5]:
fields

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[199948.348, 499949.079],
       [199948.776, 499949.471],
       [199924.434, 499973.416],
       [199925.516, 499974.575],
       [200078.598, 500117.257],
       [200087.274, 500092.176],
       [200097.218, 500070.006],
       [200107.219, 500049.151],
       [200094.465, 500037.966],
       [200092.49, 500039.734],
       [200051.237, 500002.402],
       [199970.584, 499926.843],
       [199948.348, 499949.079]]]]},
   'properties': {'year': 2016,
    'crop_code': '265',
    'crop_name': 'Grasland, blijvend',
    'fieldid': 410783,
    'area': 12959.750750893509,
    'perimeter': 536.7586864928912}}]}

In [6]:
len(fields["features"])

1

In [7]:
# for each field read the ndvi timeseries
for field in fields["features"]:
    field_id = field["properties"]["fieldid"]
    ndvi_url = "http://agrodatacube.wur.nl/api/v1/rest/fields/{0}/ndvi?fromdate={1}&todate={2}".format(field_id, year, year)
    ndvi_data = requests.get(ndvi_url, headers=header_token)
    ndvi_timeseries = ndvi_data.json()
    
    for ndvi in ndvi_timeseries["features"]:
        print(ndvi["properties"]["datum"], ndvi["properties"]["ndvi_value"])


2016-01-10 0.528658
2016-01-28 0.603808
2016-02-17 0.574633
2016-02-27 0.60895
2016-03-26 0.647033


In [8]:
# Height stats per field
for field in fields["features"]:
    field_id = field["properties"]["fieldid"]
    ahn_url = "http://agrodatacube.wur.nl/api/v1/rest/fields/{0}/ahn".format(field_id)
    ahn_data = requests.get(ahn_url, headers=header_token)
    ahn = ahn_data.json()
    print(ahn["features"][0]["properties"])
    

{'count': '19', 'area': 12959.750750893509, 'mean': 60.8421052631579, 'min': 32.0, 'max': 96.0}


In [85]:
# soiltypes per field
for field in fields["features"]:
    field_id = field["properties"]["fieldid"]
    soiltypes_url = "http://agrodatacube.wur.nl/api/v1/rest/fields/{0}/soiltypes".format(field_id)
    soiltypes_data = requests.get(soiltypes_url, headers=header_token)
    soiltypes = soiltypes_data.json()
    
    for soiltype in soiltypes["features"]:
        print(soiltype["properties"])
        soiltype_url = "http://agrodatacube.wur.nl/api/v1/rest/codes/soilcodes/{0}".format(soiltype["properties"]["soilcode"])
        soiltype_data = requests.get(soiltype_url, headers=header_token)
        soil = soiltype_data.json()
        print(soil["features"][0]["properties"])


{'fieldid': 410783, 'soilid': '40710', 'soilcode': 'Rn62Cp', 'area': 4569.349245397227, 'perimeter': 273.73633638820684}
{'soilcode': 'Rn62Cp', 'soilname': 'Zware zavel', 'soiltype': 'Rivierkleigronden'}
{'fieldid': 410783, 'soilid': '41381', 'soilcode': 'Rn95A', 'area': 8390.40150549569, 'perimeter': 400.98708543684654}
{'soilcode': 'Rn95A', 'soilname': 'Zware zavel', 'soiltype': 'Rivierkleigronden'}


In [94]:
# meteo per field
for field in fields["features"]:
    field_id = field["properties"]["fieldid"]
    meteostations_url = "http://agrodatacube.wur.nl/api/v1/rest/fields/{0}/meteostations".format(field_id)
    meteostations_data = requests.get(meteostations_url, headers=header_token)
    meteostations = meteostations_data.json()
    
    # meteostations with index 0 is the nearest to the field
    meteostation_id = meteostations["features"][0]["properties"]["meteostationid"]
    meteodata_url = "http://agrodatacube.wur.nl/api/v1/rest/meteodata?meteostation={0}&fromdate={1}&todate={2}&page_size=1000".format(meteostation_id, year, year)
    meteodata_data = requests.get(meteodata_url, headers=header_token)
    meteodata = meteodata_data.json()
    
    for meteo in meteodata["features"]:
        print(meteo["properties"])
        

{'meteostationid': 277, 'datum': '2016-01-01', 'mean_temperature': 5.4, 'min_temperature': 3.5, 'max_temperature': 8.6, 'sunshine_duration': 4.5, 'global_radiation': 267.0, 'precipitation': 0.4, 'mean_humidity': 90.0, 'max_humidity': 96.0, 'potential_evapotranspiration': 0.3}
{'meteostationid': 277, 'datum': '2016-01-02', 'mean_temperature': 3.2, 'min_temperature': 1.2, 'max_temperature': 4.4, 'sunshine_duration': 0.0, 'global_radiation': 91.0, 'precipitation': 2.3, 'mean_humidity': 88.0, 'max_humidity': 93.0, 'potential_evapotranspiration': 0.1}
{'meteostationid': 277, 'datum': '2016-01-03', 'mean_temperature': -0.5, 'min_temperature': -1.5, 'max_temperature': 1.2, 'sunshine_duration': 0.0, 'global_radiation': 63.0, 'precipitation': 0.4, 'mean_humidity': 92.0, 'max_humidity': 95.0, 'potential_evapotranspiration': 0.1}
{'meteostationid': 277, 'datum': '2016-01-04', 'mean_temperature': -3.0, 'min_temperature': -4.1, 'max_temperature': -0.5, 'sunshine_duration': 0.0, 'global_radiation': 